<a href="https://colab.research.google.com/github/otanet/Recognizing_Textual_Entailment/blob/main/%E5%A4%96%E5%9B%BD%E8%AA%9E%E3%81%AE%E3%83%87%E3%83%BC%E3%82%BF%E3%82%84%E7%9F%A5%E8%AD%98%E3%82%92%E6%97%A5%E6%9C%AC%E8%AA%9E%E3%81%AE%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E3%83%93%E3%82%B8%E3%83%8D%E3%82%B9%E3%81%AB%E6%B4%BB%E3%81%8B%E3%81%97%E3%81%BE%E3%81%97%E3%82%87%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **外国語のデータや知識を日本語の自然言語ビジネスに活かしましょう**

---

YouTubeに内容説明の動画：　https://youtu.be/X5JPdY-iZ1g

---


1.   **背景：**

* 日本語の自然言語処理は、自由に利用できるリソースがほぼ無いです。

* 京都大学のデータリソース：
http://nlp.ist.i.kyoto-u.ac.jp/EN/index.php?NLPresources

* そのほか[国立情報学研究所](https://www.nii.ac.jp/dsc/idr/datalist.html)、各企業など、承認を貰わないと使えないし、組織ではなく、個人の申請は難しいです。（現状に情報を独占し、自分を守ると考える企業が多いです。）




---



2.   **データ無い日本語NLPモデル訓練手法の紹介：**
　　
* BERTの各言語にわたって有効性が高いことに関連する情報と証拠：
>  [The Surprising Cross-Lingual Effectiveness of BERT](https://arxiv.org/pdf/1904.09077.pdf)や、[How multilingual is Multilingual BERT](https://www.aclweb.org/anthology/P19-1493.pdf)など多く論文によって、BERTの各言語にわたって有効性が高いことを証明されました。





<img src="https://drive.google.com/uc?export=view&id=15NYfWTKHTTrvJ0ZBsZiQhGRlOUaVg0nh"  width=75% >



* 言語が違うが、BERTの各言語単語トークンは多次元空間の位置がほぼ同じか、近いです。
> Sun = 太陽 = 太阳 = ดวงอาทิตย์ = 태양   (太陽に関して、多言語の太陽トークンEmbeddingが近いです。)

* 英語や中国語などのデータセットから多国語のBERTモデルを調整して、日本語の自然言語モデルに利用

* より多く外国語のデータリソースから日本語の自然言語処理モデル作成しましょう。



---



3.   **Googleさんから、モデルに多言語連携評価のベンチマーク情報**

Cross-lingual TRansfer Evaluation of Multilingual Encoders benchmark

https://sites.research.google/xtreme






In [ ]:
!pip install transformers
!pip install wget


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow_hub as hub
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
import os
import re

# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df



## **仮のビジネスタスク：映画に対し、ユーザーの感情認識**

映画に日本語の評価に関して、英語のIMDBデータソース（知識）を利用し、日本語の認識モデルを訓練しましょう。

[スタンフォード大学人工知能研究所](https://ai.stanford.edu/)のデータセット [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/)



---


**もちろん、同じく中国語、韓国語のデータ（知識）を利用し、英語や日本語のビジネス製品に展開することも出来ます。**





In [ ]:
train, test = download_and_load_datasets()

In [ ]:
train.head()

,sentence,sentiment,polarity
0,I saw this movie when I was really little. It ...,3,0
1,"In 1967, mine workers find the remnants of an ...",3,0
2,(contains slight spoilers)<br /><br />It's int...,7,1
3,"As a comic book reader, who still sees myself ...",8,1
4,"This is hardly a movie at all, but rather a re...",10,1


In [ ]:
len(train),len(test)

(25000, 25000)

In [ ]:
train.loc[0]['sentence']

'I saw this movie when I was really little. It is, by far, one of the strangest movies I have ever seen. Now, normally, I like weird movies, but this was just a bit too much.<br /><br />There\'s not much of a plot to the movie. If anything, it starts out like Toy Story, where toys come to life, and Raggedy Ann and Andy go on an adventure to rescue their new friend, Babette. From there, craziness ensues. There\'s the Greedy, the Looneys, a sea monster named Gazooks, and a bunch of pirates singing show tunes, all of which just made the movie weirder. Also, I can\'t help but feel that Babette is annoying and a bit too whiny. She definitely didn\'t help the movie.<br /><br />Now, even though I didn\'t like this movie, there were a few cute parts. I liked the camel\'s song. Even though it was a song about being lonely, it had a friendly feel to it. Then, there was Sir Leonard. While most of the Looneys were just plain nuts, Sir Leonard was the most interesting and probably the funniest. Kin

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification,AdamW,BertConfig


BERT多国語モデルを利用します。 (104国語)

いろいろ国の単語と文字の例：

 'ұ', 'Ҳ', 'ҳ', 'Ҷ', 'ҷ', 'Һ', 'һ', 'Ӏ', 'ӑ', 'ӗ', 'Ә', 'ә', 'ӣ', 'Ө', 'ө', 'Ӯ', 'ӯ', 'ӳ', 'Ա', 'Բ', 'Գ', 'Դ', 'Ե', 'Զ', 'Է', 'Ը'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
# tokenizer.vocab.keys()

In [ ]:
tokenizer.vocab_size

119547

もちろん、日本語の文字が入っています。


In [ ]:
tokenizer.tokenize('意味のないアラート発動だった。')

['意', '味', 'のない', '##ア', '##ラー', '##ト', '発', '動', 'だった', '。']

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize('意味のないアラート発動だった。'))

[3906, 2824, 82181, 18226, 29004, 13913, 5712, 2621, 19101, 1882]

In [ ]:
tokenizer.encode('意味のないアラート発動だった。', add_special_tokens=True)

[101, 3906, 2824, 82181, 18226, 29004, 13913, 5712, 2621, 19101, 1882, 102]

## **IMDBのデータ前処理**

In [ ]:
sentences = train.sentence.values
labels = train.polarity.values

In [ ]:
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 256,          
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])    
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  I saw this movie when I was really little. It is, by far, one of the strangest movies I have ever seen. Now, normally, I like weird movies, but this was just a bit too much.<br /><br />There's not much of a plot to the movie. If anything, it starts out like Toy Story, where toys come to life, and Raggedy Ann and Andy go on an adventure to rescue their new friend, Babette. From there, craziness ensues. There's the Greedy, the Looneys, a sea monster named Gazooks, and a bunch of pirates singing show tunes, all of which just made the movie weirder. Also, I can't help but feel that Babette is annoying and a bit too whiny. She definitely didn't help the movie.<br /><br />Now, even though I didn't like this movie, there were a few cute parts. I liked the camel's song. Even though it was a song about being lonely, it had a friendly feel to it. Then, there was Sir Leonard. While most of the Looneys were just plain nuts, Sir Leonard was the most interesting and probably the funniest.

In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

22,500 training samples
2,500 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

#batch_size = 32
batch_size = 8

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

## **英語のIMDBデータセットから、多国語NLPモデルに訓練する。**

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup
# epochs = 2
epochs = 1

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()

for epoch_i in range(0, epochs):
    # Training
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # Validation
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        

            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 1 ========
Training...
  Batch    40  of  2,813.    Elapsed: 0:00:09.
  Batch    80  of  2,813.    Elapsed: 0:00:18.
  Batch   120  of  2,813.    Elapsed: 0:00:28.
  Batch   160  of  2,813.    Elapsed: 0:00:37.
  Batch   200  of  2,813.    Elapsed: 0:00:46.
  Batch   240  of  2,813.    Elapsed: 0:00:55.
  Batch   280  of  2,813.    Elapsed: 0:01:04.
  Batch   320  of  2,813.    Elapsed: 0:01:13.
  Batch   360  of  2,813.    Elapsed: 0:01:23.
  Batch   400  of  2,813.    Elapsed: 0:01:32.
  Batch   440  of  2,813.    Elapsed: 0:01:41.
  Batch   480  of  2,813.    Elapsed: 0:01:50.
  Batch   520  of  2,813.    Elapsed: 0:01:59.
  Batch   560  of  2,813.    Elapsed: 0:02:08.
  Batch   600  of  2,813.    Elapsed: 0:02:18.
  Batch   640  of  2,813.    Elapsed: 0:02:27.
  Batch   680  of  2,813.    Elapsed: 0:02:36.
  Batch   720  of  2,813.    Elapsed: 0:02:45.
  Batch   760  of  2,813.    Elapsed: 0:02:54.
  Batch   800  of  2,813.    Elapsed: 0:03:03.
  Batch   840  of

## **英語のデータセットから訓練した効果を確認しましょう。**

In [ ]:
model.eval()

def sa_function(text):
    encoded_dict = tokenizer.encode_plus(
                        text,   
                        add_special_tokens = True,
                        max_length = 256,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids = encoded_dict['input_ids'].to(device)
    outputs = model(input_ids)
    negative = outputs[0][0][0].item()
    positive = outputs[0][0][1].item()
    positive_p = np.exp(positive)/(np.exp(positive)+np.exp(negative))
    negative_p = np.exp(negative)/(np.exp(positive)+np.exp(negative))
    if positive_p>=negative_p:
        result = {"label" : "ポジティブ","score" : positive_p}
    else:
        result = {"label" : "ネガティブ","score" : negative_p}
            
    return result

* Yahoo映画サイトの[新着・作品ユーザーレビュー](https://movies.yahoo.co.jp/review/)　https://movies.yahoo.co.jp/review/

* 映画.COMからの [映画レビュー人気作品ランキング](https://eiga.com/movie/review/ranking/)　https://eiga.com/movie/review/ranking/


---


**以上二つ映画評価サイトからユーザーのコメントをコピーして、英語の微調整するモデルに試してください 。**

In [ ]:
test = """
映画でピエロと言えばホラー、ゾンビと言えばコメディというイメージが定着している中でまさかのゾンビ社会派ドラマ！？

しかしこれがなかなか巧みな設定で、ゾンビも元々普通の人間、噛まれてゾンビ化してるけど死んでるわけじゃない、というわけで薬による治療も可能という世界のお話。
ここでさらに回復後もゾンビの時の記憶は残っているという設定が心理描写の伸びしろを生み出していて、これらの新しい着眼点を存分に生かした娯楽作品になっています。

私個人的には非感染者からの元ゾンビへの憎悪と迫害、そこからの反発を、現実の移民問題等と結びつけて何か伝えようという意図は感じなかったので社会派ドラマという見え方はしなかったです。

みんな大好きなゾンビという古典クリーチャーを本質に沿いつつ深堀りできた事と、それが効果的に面白さにつながってる点にクリエーターの意識の高さを感じる秀作だと思いました。
"""


sa_function(test)


{'label': 'ポジティブ', 'score': 0.7129202653290919}

In [ ]:
test = """
レビューは低評価が多いのであまり期待しないで鑑賞。やはり低評価が頷ける内容でした(途中久々に睡魔が、、)。

カンバーバッチ主演でこの出来はかなりもったいない！もう少しどうにかならなかったのか･･残念過ぎる。まずエジソンとウェスティングハウスのそれぞれの妻のエピソードは要らないだろう。完全に蛇足、時間の無駄。家族の話は全カットしてエジソン vs ウェスティングハウスのビジネス物に徹するべきだったのではないか？下手に家族の話を入れた事によって全体がぼやけた印象。特にウェスティングハウスの妻の"私頭いいのよ"的な話し方や態度が鼻についた。夫の威を借る妻･･。話の本筋に全く関係ない妻のエピソードに時間を割いてテスラの話をほとんど描かないのは理解不可能。テスラの天才ぶりが全く伝わってこず、ただ要領の悪い若者みたいになってしまって気の毒。

専門的な用語等は言葉だけでなく、視覚での説明･解説があればかなり分かりやすくなっただろうと思う。エジソンとウェスティングハウスがずっといがみ合い･足の引っ張り合いをしているばかりで、成功して"ヨッシャー👍！！"みたいに歓喜で盛り上がりがる場面が無いので単調に感じる。これではヒットは難しく興収的にかなり厳しいだろう。
もう少し工夫すれば良作になる題材だったと思うので本当にもったいない。

確かに邦題もイマイチですね。そのまま"電流戦争"でもいい気がする。
カンバーバッチならイミテーション･ゲームの方が断然おもしろかった。
"""


sa_function(test)

{'label': 'ネガティブ', 'score': 0.9829522138774879}

中国語の映画評価を、この英語微調整するモデルで効果を確認しましょう。

[豆瓣电影](https://movie.douban.com/)



In [ ]:
#test = "这个影片拍的非常差。"
#test = "这个影片拍的很好。"
test = """
这个影片拍的有些故弄玄虚了。神神叨叨一个小时后揭露女主真实身份，随后便是男主急匆匆地一路反击，哪怕他前一分钟精神再恍惚，也能凭借强大的主角光环不再恍惚而硬扛到最后。反转太生硬，扯上邪教其实也并不是什么新鲜题材了。最后20多分钟太仓促，看的时候感觉像是导演在提示大家：搞快点，电影马上就结束了，别在乎什么逻辑了，能勉强圆回来就已经够意思啦。非常差。
"""


sa_function(test)

{'label': 'ネガティブ', 'score': 0.9841572709176958}

## 総論： 人工知能のニューラルネットワークと生物の脳


一つ多言語NLPモデルは　人間のように　ニューラルネットワークが　知識を分かれば　多言語に渡って利用することが出来ます。

* より多く外国語のデータリソースや知識から日本ビジネスを完成しましょう。